<a href="https://colab.research.google.com/github/lucabem/CompeticionMineria/blob/main/Presentacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Cargar paquetes

In [1]:

!pip install --upgrade tensorflow_hub
!pip install --upgrade tensorflow_addons

import os
import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing import image
import numpy as np
import pandas as pd
from google.colab import files
import tensorflow_addons as tfa
import pathlib



Requirement already up-to-date: tensorflow_hub in /usr/local/lib/python3.7/dist-packages (0.12.0)
     |████████████████████████████████| 706kB 6.8MB/s 


# 2. Cargar modelo

### Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/ModeloFinal/resnet50_ms_transfer_final.17-0.950.hdf5')

### Manual

In [5]:
files.upload()

{}

In [ ]:
!unzip "model.zip"

In [ ]:
ls model_2048_512_1024/

In [ ]:
path_to_model = "/content/model_2048_512_1024"
model = tf.keras.models.load_model(path_to_model, compile=True, custom_objects={'f1':tfa.metrics.F1Score(num_classes=25, average='weighted')})

### Modelo

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 1024)        18875392  
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 512)         4719104   
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2048)              9439232   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)              

## 3. Cargar el Dataset

Se carga manualmente y se descomprime.

In [10]:
files.upload()

Saving test.zip to test.zip


In [ ]:
! unzip "test.zip"

# 4. Predicción

Definir función que haga las predicciones con el modelo sobre el set de test.

In [ ]:
mkdir test

In [27]:
mv Test-definitivo-de-Kaggle-03:2021/* test

In [28]:
path = 'test'

In [29]:
img_width = 224
img_height = 224

In [23]:
def make_predictions(model,
                     path_test ='test',
                     name_file_submission = 'submission.csv'):

  images_test = os.listdir(path_test)
  samples_to_predict = []

  for img in images_test:
    path_img = os.path.join(path_test, img)
    img = image.load_img(path_img,
                        target_size=(img_width, img_height))
    x = image.img_to_array(img)
    samples_to_predict.append(x)

  samples_to_predict  = np.array(samples_to_predict)
  predictions = model.predict(samples_to_predict)
  classes = np.argmax(predictions, axis = 1)

  data = {'id.jpg': [img for img in images_test], 'label': classes}
  data = pd.DataFrame(data)
  data.to_csv(name_file_submission, index = False)

  return data



Hacer la predicción.


In [30]:
make_predictions(model, 
                 path_test=path)

,id.jpg,label
0,Urban_100_9.jpg,24
1,Grassland__13.jpg,17
2,Forest_CDB_9.jpg,5
3,Continenta_19.jpg,3
4,Close_Shru_11.jpg,2
...,...,...
495,Marine_Wat_15.jpg,18
496,Cropland_s_4.jpg,4
497,Grassland__1.jpg,17
498,Forest_DEB_4.jpg,11


# 5. Descarga de resultados

In [31]:
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>